## Result
- [Inference Code](https://www.kaggle.com/code/shelterw/sft-llama-3-8b-inference)    

- [Base Model: llama-3-8b-Instruct-bnb-4bit](https://huggingface.co/unsloth/llama-3-8b-Instruct-bnb-4bit)

| subset | log loss |
| - | - |
| Eval | 0.9231|
| LB | 0.936 |

## Note
If you want to reproduce the code, please note the following:
- use all data
- set per_device_train_batch_size=4
- 1 epoch using A10 took ~15h


In [18]:
import os

from google.colab import drive

INIT=True
PATH="/content"

if INIT:
  drive.mount('/content/drive')
  ! pip install kaggle

  !mkdir ~/.kaggle


  ! cp kaggle.json ~/.kaggle/
  ! chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [19]:
!pip install datasets
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

In [46]:
WEIGHTS_PATH = "/content/drive/MyDrive/IA/llama/checkpoint-7600"

In [21]:
import os
import copy
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from datasets import Dataset
from scipy.special import softmax
from sklearn.preprocessing import LabelEncoder
from transformers import (
    BitsAndBytesConfig,
    LlamaPreTrainedModel,
    LlamaModel,
    AutoTokenizer,
    PreTrainedTokenizerBase,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
from transformers.modeling_outputs import CausalLMOutputWithPast
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from sklearn.metrics import log_loss, accuracy_score

In [22]:
!kaggle competitions download -c lmsys-chatbot-arena

lmsys-chatbot-arena.zip: Skipping, found more recently modified local copy (use --force to force download)


In [23]:
! unzip -o  lmsys-chatbot-arena.zip -d  lmsys-chatbot-arena

Archive:  lmsys-chatbot-arena.zip
  inflating: lmsys-chatbot-arena/sample_submission.csv  
  inflating: lmsys-chatbot-arena/test.csv  
  inflating: lmsys-chatbot-arena/train.csv  


### Configurations

In [24]:
TRAIN_CSV = "lmsys-chatbot-arena/train.csv"
model_path = "unsloth/llama-3-8b-Instruct-bnb-4bit"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 2400
BATCH_SIZE = 2
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']
columns_to_vectorize = ["prompt", "response_a", "response_b"]

train = pd.read_csv(TRAIN_CSV)
#train = train.head(100)
train['label'] = train[target_columns].idxmax(axis=1)
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])
train = train[columns_to_vectorize + ['label']]

In [25]:
train

,prompt,response_a,response_b,label
0,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",0
1,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",1
2,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",2
3,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",0
4,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",1
...,...,...,...,...
57472,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",0
57473,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",0
57474,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",0
57475,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",1


### Tokenizer and prepare dataset, metrics

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.add_eos_token = True
tokenizer.padding_side = 'right'

LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]

def tokenize(example, tokenizer):
    prompt = tokenizer('<prompt>: ' + " ".join(eval(example['prompt'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_a = tokenizer('\n\n<response_a>: ' + " ".join(eval(example['response_a'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_b = tokenizer('\n\n<response_b>: ' + " ".join(eval(example['response_b'], {"null": ""})), add_special_tokens=False)["input_ids"]
    if len(prompt+response_a+response_b) > MAX_LENGTH:
        prompt = tokenizer('<prompt>: ' + eval(example['prompt'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:256]
        response_a = tokenizer('\n\n<response_a>: ' + eval(example['response_a'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
        response_b = tokenizer('\n\n<response_b>: ' + eval(example['response_b'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
    extra_prompt = tokenizer('\n\n---------\nWhich is the better response for the prompt ? a or b or tie ?\n\nAnswer: ', add_special_tokens=False)["input_ids"]

    label_token_id = LABEL_IDS[int(example['label'])]
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + [label_token_id] + [tokenizer.eos_token_id]
    attention_mask = len(input_ids)*[1]
    labels = [-100]* len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + [label_token_id] + [tokenizer.eos_token_id]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


In [27]:
def load_data(df, tokenizer):
    raw_datasets = Dataset.from_pandas(df)
    tokenized_datasets = raw_datasets.map(
        tokenize,
        remove_columns=raw_datasets.column_names,
        fn_kwargs={'tokenizer': tokenizer}
    )
    return tokenized_datasets

def compute_metrics(pred):
    logits, labels = pred
    preds = logits.argmax(axis=-1)
    label_tokens_ids = np.array(LABEL_IDS)
    index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}
    labels = labels[np.isin(labels, label_tokens_ids)]
    labels = np.array([index_mapping[label.item()] for label in labels])
    acc = accuracy_score(labels, preds)
    probs = softmax(logits, axis=-1)
    log_loss_ = log_loss(labels, probs)
    return {'accuracy': acc, 'log_loss': log_loss_}

n_splits = 5
fold_idx = 0
ds = load_data(train, tokenizer)
folds = [
    (
        [i for i in range(len(ds)) if i % n_splits != fold_idx],
        [i for i in range(len(ds)) if i % n_splits == fold_idx]
    )
    for fold_idx in range(n_splits)
]
train_idx, eval_idx = folds[fold_idx]

Map:   0%|          | 0/57477 [00:00<?, ? examples/s]

In [28]:
 eval_idx = eval_idx [:300]

In [29]:
train.shape

(57477, 4)

In [30]:
len(ds )

57477

In [31]:
len(train_idx)

45981

### Model

In [32]:
class Llama3ForSFT(LlamaPreTrainedModel):
    _tied_weights_keys = ["lm_head.weight"]
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        self.post_init()

    def forward(
        self,
        input_ids= None,
        attention_mask= None,
        position_ids = None,
        past_key_values= None,
        inputs_embeds= None,
        labels= None,
        use_cache= None,
        output_attentions= None,
        output_hidden_states = None,
        return_dict= None,
        cache_position = None,
    ):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        hidden_states = outputs[0]
        if self.config.pretraining_tp > 1:
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
            logits = torch.cat(logits, dim=-1)
        else:
            logits = self.lm_head(hidden_states)
        logits = logits.float()

        loss = None
        if labels is not None:
            # Shift so that tokens < n predict n
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = nn.CrossEntropyLoss()
            shift_logits = shift_logits.view(-1, self.config.vocab_size)
            shift_labels = shift_labels.view(-1)
            # Enable model parallelism
            shift_labels = shift_labels.to(shift_logits.device)

            label_tokens_ids = torch.tensor(LABEL_IDS,device=shift_labels.device)
            index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}
            true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]
            true_labels = torch.tensor([index_mapping[label.item()] for label in true_labels], device=true_labels.device)
            true_logits = shift_logits[torch.isin(shift_labels, label_tokens_ids)][:,label_tokens_ids]
            loss = loss_fct(true_logits, true_labels)

        return CausalLMOutputWithPast(
            loss=loss,
            logits=true_logits,
        )

In [48]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM,
    target_modules=['q_proj', 'k_proj', 'v_proj',],
)

model = Llama3ForSFT.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
)




Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [49]:
model = PeftModel.from_pretrained(model, model_id=WEIGHTS_PATH)

In [35]:
batch_size=BATCH_SIZE
max_length=MAX_LENGTH

In [55]:
a_win, b_win, tie = [], [], []

In [37]:
df = ds.select(eval_idx).to_pandas()

In [38]:
df

,input_ids,attention_mask,labels
0,"[128000, 8085, 15091, 27916, 2209, 433, 57323,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
1,"[128000, 8085, 15091, 27916, 19196, 264, 7477,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
2,"[128000, 8085, 15091, 27916, 3350, 264, 10344,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
3,"[128000, 8085, 15091, 27916, 1148, 656, 499, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
4,"[128000, 8085, 15091, 27916, 9842, 264, 10344,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
...,...,...,...
295,"[128000, 8085, 15091, 27916, 3639, 596, 279, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
296,"[128000, 8085, 15091, 27916, 3639, 374, 279, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
297,"[128000, 8085, 15091, 27916, 3371, 757, 810, 9...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
298,"[128000, 8085, 15091, 27916, 1796, 279, 73095,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."


In [39]:
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
from tqdm import tqdm

In [56]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Llama3ForSFT(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [57]:


for start_idx in tqdm(range(0, len(df), batch_size)):
    end_idx = min(start_idx + batch_size, len(df))
    tmp = df.iloc[start_idx:end_idx]
    input_ids = tmp["input_ids"].to_list()
    attention_mask = tmp["attention_mask"].to_list()
    labels = tmp["labels"].to_list()
    inputs = pad_without_fast_tokenizer_warning(
        tokenizer,
        {"input_ids": input_ids, "attention_mask": attention_mask},
        padding="longest",
        pad_to_multiple_of=None,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    pad_labels=[]
    for label in labels:
        label = list(label) + [tokenizer.pad_token_id]*(input_ids[0].shape[0]-label.shape[0])
        pad_labels.append(label)
    labels = torch.tensor(pad_labels).to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    proba = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
    a_win.extend(proba[:, 0].tolist())
    b_win.extend(proba[:, 1].tolist())
    tie.extend(proba[:, 2].tolist())


100%|██████████| 150/150 [02:11<00:00,  1.14it/s]


In [58]:
df

,input_ids,attention_mask,labels
0,"[128000, 8085, 15091, 27916, 2209, 433, 57323,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
1,"[128000, 8085, 15091, 27916, 19196, 264, 7477,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
2,"[128000, 8085, 15091, 27916, 3350, 264, 10344,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
3,"[128000, 8085, 15091, 27916, 1148, 656, 499, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
4,"[128000, 8085, 15091, 27916, 9842, 264, 10344,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
...,...,...,...
295,"[128000, 8085, 15091, 27916, 3639, 596, 279, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
296,"[128000, 8085, 15091, 27916, 3639, 374, 279, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
297,"[128000, 8085, 15091, 27916, 3371, 757, 810, 9...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."
298,"[128000, 8085, 15091, 27916, 1796, 279, 73095,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10..."


In [60]:
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
log_loss( train.loc[eval_idx,"label"],  np.array([a_win, b_win, tie]).transpose().reshape(-1,3)), accuracy_score( train.loc[eval_idx,"label"],  np.array([a_win, b_win, tie]).transpose().reshape(-1,3).argmax(axis=1))

(0.9287147597336567, 0.5633333333333334)